In [12]:
# ! pip install slack_sdk

In [6]:
import requests
import xml.etree.ElementTree as ET
from slack_sdk import WebClient
import os
import re
from zoneinfo import ZoneInfo
from datetime import datetime, timedelta

SLACK_TOKEN = os.environ["SLACK_TOKEN"]  # get token from environment

# Configuration
URL = "https://calendar.usc.edu/calendar/1.xml"
CHANNEL = "#events-rss"  # or channel ID

client = WebClient(token=SLACK_TOKEN)
tomorrow = datetime.now(ZoneInfo("America/Los_Angeles")).date() + timedelta(days=1)

# Fetch the XML feed
response = requests.get(URL)
response.raise_for_status()
xml_data = response.content
root = ET.fromstring(xml_data)

events_today = []

for item in root.findall(".//item"):
    title = item.findtext("title") or "No title"
    link = item.findtext("link") or ""
    pub_date = item.findtext("{http://purl.org/dc/elements/1.1/}date") or item.findtext("pubDate")

    if pub_date:
        try:
            # Parse ISO-style (<dc:date>) or <pubDate> style
            if "T" in pub_date:
                event_date = datetime.date.fromisoformat(pub_date.split("T")[0])
            else:
                event_date = datetime.datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z").date()

            if event_date == tomorrow:
                # Remove date prefix (e.g., "Nov 4, 2025:" or "November 4, 2025:")
                title = re.sub(r"^[A-Za-z]{3,9}\s+\d{1,2},\s+\d{4}:\s*", "", title)
                events_today.append(f"{title}\n{link}")
        except Exception as e:
            print("Bad date format:", pub_date, e)

# Post to Slack
if events_today:
    formatted_date = tomorrow.strftime("%b %d, %Y")
    border = "────────────────────────────────────────────────────────────"
    message = (
        f"{border}\n"
        f"*📅 USC Events Happening Tomorrow ({formatted_date}):*\n\n"
        + "\n"
        + "\n".join(f"• {e}" for e in events_today)
        + "\n"
        + f"\n\n_USC Events Happening Tomorrow ({formatted_date})_"
        + f"\n\n{border}"
    )
    resp = client.chat_postMessage(channel=CHANNEL, text=message)
    print(resp)
else:
    print("No events today.")

KeyError: 'SLACK_TOKEN'